In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the PINN model
class PINNInventoryModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PINNInventoryModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.network(x)

# Define custom loss function for PINNs in inventory management
class PINNLoss(nn.Module):
    def __init__(self, lambda_physics=1.0, lambda_cost=0.1):
        super(PINNLoss, self).__init__()
        self.lambda_physics = lambda_physics
        self.mse_loss = nn.MSELoss()
    
    def forward(self, pred_inventory, true_inventory, past_inventory, supply, demand, order=None, lead_time=None, alpha=0.0, backlog_penalty=0.1):
        # Data loss (MSE loss between predicted and actual inventory levels)
        data_loss = self.mse_loss(pred_inventory, true_inventory)
        
        # Physics-based loss: Inventory balance constraint
        inventory_constraint = torch.mean((pred_inventory - (past_inventory + supply - demand)) ** 2)
        
        # Replenishment delay constraint (only if order and lead_time are provided)
        replenishment_constraint = torch.tensor(0.0, device=pred_inventory.device)
        if order is not None and lead_time is not None:
            replenishment_constraint = torch.mean((supply - torch.roll(order, shifts=int(lead_time))) ** 2)
        
        # Enforce non-negative inventory (stockout penalty)
        stockout_penalty = torch.mean(torch.clamp(-pred_inventory, min=0) ** 2) * backlog_penalty
        
        # Deterioration penalty (if applicable)
        deterioration_penalty = torch.mean((pred_inventory - (true_inventory - pred_inventory * alpha)) ** 2)
        
        # Total loss
        total_loss = data_loss + self.lambda_physics * (inventory_constraint + replenishment_constraint + deterioration_penalty + stockout_penalty)
        return total_loss

# Hyperparameters
input_dim = 3  # Inputs: past inventory, supply, demand
hidden_dim = 16
output_dim = 1  # Future inventory prediction
learning_rate = 0.01
lambda_physics = 0.5
backlog_penalty = 0.1  # Penalty for negative inventory

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Model and Optimizer
model = PINNInventoryModel(input_dim, hidden_dim, output_dim).to(device)
criterion = PINNLoss(lambda_physics=lambda_physics).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Dummy dataset for training (Replace with real data)
n_samples = 1000
past_inventory = torch.rand(n_samples, 1).to(device) * 100
supply = torch.rand(n_samples, 1).to(device) * 50
demand = torch.rand(n_samples, 1).to(device) * 30
true_future_inventory = past_inventory + supply - demand
order = torch.rand(n_samples, 1).to(device) * 50  # Dummy order values
lead_time = 1  # Assumed lead time (can be varied)
alpha = 0.02  # Example deterioration rate

# Training Loop
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    inputs = torch.cat((past_inventory, supply, demand), dim=1)
    predictions = model(inputs)
    loss = criterion(predictions, true_future_inventory, past_inventory, supply, demand, order, lead_time, alpha, backlog_penalty)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

print("Training complete.")


Epoch 0, Loss: 10906.4492
Epoch 10, Loss: 6162.9912
Epoch 20, Loss: 881.5643
Epoch 30, Loss: 277.7636
Epoch 40, Loss: 292.5543
Epoch 50, Loss: 246.8040
Epoch 60, Loss: 201.1535
Epoch 70, Loss: 205.3310
Epoch 80, Loss: 199.9096
Epoch 90, Loss: 199.7055
Training complete.
